<a href="https://colab.research.google.com/github/PhilipMathieu/unet-orthoimagery/blob/main/colab_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-Net Training Notebook

In [1]:
!pip install wandb
!rm -r sample_data
!git clone --single-branch --branch philip https://github.com/PhilipMathieu/unet-orthoimagery.git
!mv unet-orthoimagery/* ./

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=6ec322c49d1925ff2a785db149d45abaaa40c81e9bff30d39d0c6bc698287837
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools
Cloning into 'unet-orthoimagery'...
remote: Enumerating objects: 104922, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 104922 (delta 48), reused 55 (delta 21), pack-reused 104825


In [3]:
import sys
sys.path.append('./src/')

[WandB Quickstart](https://docs.wandb.ai/quickstart#:~:text=Provide%20your%20API%20key%20when%20prompted.) (including link to your API key, if logged in)

In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: philipmathieu (unet-ortho). Use `wandb login --relogin` to force relogin


True

In [5]:
import logging
import torch
from src.train import train_model
from src.unet.unet_model import UNet

In [6]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device: {device}')
print(device)

cuda


In [7]:
model = UNet(n_channels=4, n_classes=1, bilinear=False)
model = model.to(memory_format=torch.channels_last)

In [8]:
logging.info(f'Network:\n'
                 f'\t{model.n_channels} input channels\n'
                 f'\t{model.n_classes} output channels (classes)\n'
                 f'\t{"Bilinear" if model.bilinear else "transposed conv"} upscaling')


model.to(device=device);

In [9]:
data_dir = "data/Image_Chips_128_overlap_balanced_dem/"

In [ ]:
try:
    train_model(
        model=model,
        epochs=50,
        device=device,
        data_dir=data_dir,
        batch_size=128,
        img_scale=1.0
    )
except torch.cuda.OutOfMemoryError: # Giving me syntax error saying '"OutOfMemoryError" is not a valid exception class.'
    logging.error('Detected OutOfMemoryError! '
                    'Enabling checkpointing to reduce memory usage, but this slows down training. '
                    'Consider enabling AMP (--amp) for fast and memory efficient training')
    torch.cuda.empty_cache()
    model.use_checkpointing()
    train_model(
        model=model,
        epochs=100,
        device=device,
        data_dir=data_dir
    )

  0%|          | 0/640 [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
learning rate,██▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train loss,▅▃▅▄▁▄▄▃▄▅▄▂▃▇▂▇▁▂▄▇▅▆▄▂▅▂▆▅▅▅▄▃█▃▃▂▆▄█▄
validation Dice,▁▆▆▇████████████████████████████████████
epoch,20
learning rate,0.0
step,260
train loss,1.23739
validation Dice,0.46148


Overall:   0%|          | 0/50 [00:00<?, ?epochs/s]

Epoch 1/50:   0%|          | 0/427 [00:00<?, ?img/s]

Epoch 2/50:   0%|          | 0/427 [00:00<?, ?img/s]

Epoch 3/50:   0%|          | 0/427 [00:00<?, ?img/s]

Epoch 4/50:   0%|          | 0/427 [00:00<?, ?img/s]